### 블로그 크롤링 수정

#### 기존의 셀레니움 코드 사용시 몇몇의 블로그 본문이 읽히지않는 문제 발생

#### 셀레니움으로 url을 불러온 후 requsts를 사용하는 코드로 수정

### 페이지들의 링크 가져오기

In [5]:
import pandas as pd
from tqdm import tqdm
import math
from selenium import webdriver
# 빈창 만들기
options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')
options.add_argument('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36')

driver = webdriver.Chrome('chromedriver', options = options)
url_list = []
title=[]
name=[]
day=[]
content_list = ""
text = "전기차 운행기"

for i in tqdm(range(1, math.ceil(70/7)+1)):  # 블로그 건 수를 기준으로 블로그 내용을 읽어옴
    url = 'https://section.blog.naver.com/Search/Post.naver?pageNo='+ str(i) +\
    '&rangeType=PERIOD&orderBy=sim&startDate=2016-01-01&endDate=2021-08-04&keyword=' + text
    driver.get(url)
    time.sleep(1)
 
    for j in range(1, 8): # 각 블로그 주소 저장(8개로!)
        url_titles = driver.find_element_by_css_selector('#content > section > div.area_list_search > div:nth-child({}) > div > div.info_post > div.desc > a.desc_inner'.format(j))
        url_title = url_titles.get_attribute('href')
        url_list.append(url_title)

        titles=driver.find_element_by_css_selector('#content > section > div.area_list_search > div:nth-child({}) > div > div.info_post > div.desc > a.desc_inner > strong > span'.format(j)).text
        title.append(titles)
        
        names=driver.find_element_by_css_selector('#content > section > div.area_list_search > div:nth-child({}) > div > div.info_post > div.writer_info > a > em'.format(j)).text
        name.append(names)

        days=driver.find_element_by_css_selector('#content > section > div.area_list_search > div:nth-child({}) > div > div.info_post > div.writer_info > span.date'.format(j)).text
        day.append(days)
                       
        
print("url 수집 끝, 해당 url 데이터 크롤링")

df_url = pd.DataFrame()
df_url['url'] = url_list
df_url['titles'] = title
df_url['names'] = name
df_url['days'] = day

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:19<00:00,  1.93s/it]

url 수집 끝, 해당 url 데이터 크롤링


### 블로그 크롤링

#### 기능, 번역보기 이후로 본문이 존재함 -> 정리

In [87]:
import requests
from bs4 import BeautifulSoup
import time

def make_iframe(url) :
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36"}
    res = requests.get(url, headers=headers)
    res.raise_for_status() # 문제시 프로그램 종료
    soup = BeautifulSoup(res.text, "lxml")
    
    src_url = "https://blog.naver.com/" + soup.iframe["src"]
    time.sleep(1)
    response = requests.get(src_url)
    time.sleep(1)
    logno = src_url.find('logNo=')
    red= src_url.find('&red')
    log_no =src_url[logno+6 : red]
    
    if response.status_code == 200:
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')
        source = '#post-view{}'.format(log_no)
        if soup.select_one(source) == [] :
            pass  
        else :
            title = soup.select_one(source).text.replace('\n','')
        if title.find('기능번역보기') != -1 :
            del_idx = title.find('기능번역보기')
            title = title[del_idx +6:]
        else :
            title = title
    print("-"*6,'진행중',"-"*6)
    return title

In [6]:
df_url['리뷰'] = df_url['url'].apply(lambda x : make_iframe(x))

------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중

### 파일 저장

#### encoding 문제 발생(해결)

In [10]:
import pandas as pd
writer = pd.ExcelWriter(r'전기차 운행기.xlsx', engine='xlsxwriter',options={'strings_to_urls': False})
df_url.to_excel(writer)
writer.close()

<ipython-input-10-3dd68f0cd6e8>:2: FutureWarning: Use of **kwargs is deprecated, use engine_kwargs instead.
  writer = pd.ExcelWriter(r'전기차 운행기.xlsx', engine='xlsxwriter',options={'strings_to_urls': False})


In [12]:
import pandas as pd
from tqdm import tqdm
import math
from selenium import webdriver
# 빈창 만들기
options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')
options.add_argument('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36')

driver = webdriver.Chrome('chromedriver', options = options)
url_list = []
title=[]
name=[]
day=[]
content_list = ""
text = "전기차 사용기"

for i in tqdm(range(1, math.ceil(524/7))):  # 1~2페이지까지의 블로그 내용을 읽어옴
    url = 'https://section.blog.naver.com/Search/Post.naver?pageNo='+ str(i) +\
    '&rangeType=PERIOD&orderBy=sim&startDate=2016-01-01&endDate=2021-08-04&keyword=' + text
    driver.get(url)
    time.sleep(1)
 
    for j in range(1, 8): # 각 블로그 주소 저장(8개로!)
        url_titles = driver.find_element_by_css_selector('#content > section > div.area_list_search > div:nth-child({}) > div > div.info_post > div.desc > a.desc_inner'.format(j))
        url_title = url_titles.get_attribute('href')
        url_list.append(url_title)

        titles=driver.find_element_by_css_selector('#content > section > div.area_list_search > div:nth-child({}) > div > div.info_post > div.desc > a.desc_inner > strong > span'.format(j)).text
        title.append(titles)
        
        names=driver.find_element_by_css_selector('#content > section > div.area_list_search > div:nth-child({}) > div > div.info_post > div.writer_info > a > em'.format(j)).text
        name.append(names)

        days=driver.find_element_by_css_selector('#content > section > div.area_list_search > div:nth-child({}) > div > div.info_post > div.writer_info > span.date'.format(j)).text
        day.append(days)
                       
        
print("url 수집 끝, 해당 url 데이터 크롤링")

df_url = pd.DataFrame()
df_url['url'] = url_list
df_url['titles'] = title
df_url['names'] = name
df_url['days'] = day

df_url['리뷰'] = df_url['url'].apply(lambda x : make_iframe(x))

100%|██████████████████████████████████████████████████████████████████████████████████| 74/74 [02:04<00:00,  1.68s/it]


url 수집 끝, 해당 url 데이터 크롤링
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
---

------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중

<ipython-input-12-8b2455b5f7b0>:49: FutureWarning: Use of **kwargs is deprecated, use engine_kwargs instead.
  writer = pd.ExcelWriter(r'전기차 사용기.xlsx', engine='xlsxwriter',options={'strings_to_urls': False})


UnicodeEncodeError: 'utf-8' codec can't encode characters in position 2735-2736: surrogates not allowed

#### encoding 문제 해결

In [20]:
df_copy = df_url.copy()

In [15]:
def encode_error(x) :
    x = x.encode('utf-8', 'replace').decode()
    return x

In [21]:
df_copy['리뷰'] = df_url.리뷰.apply(lambda x : encode_error(x))

In [18]:
writer = pd.ExcelWriter(r'전기차 사용기.xlsx', engine='xlsxwriter',options={'strings_to_urls': False})
df_copy.to_excel(writer)
writer.close()

<ipython-input-18-9be5a5676a8f>:1: FutureWarning: Use of **kwargs is deprecated, use engine_kwargs instead.
  writer = pd.ExcelWriter(r'전기차 사용기.xlsx', engine='xlsxwriter',options={'strings_to_urls': False})


In [ ]:
writer = pd.ExcelWriter(r'전기차 사용기.xlsx', engine='xlsxwriter',options={'strings_to_urls': False})
df_url.to_excel(writer)
writer.close()

In [94]:
import pandas as pd
from tqdm import tqdm
import math
from selenium import webdriver
# 빈창 만들기
options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')
options.add_argument('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36')

driver = webdriver.Chrome('chromedriver', options = options)
url_list = []
title=[]
name=[]
day=[]
content_list = ""
text = "전기차 후기"

for i in tqdm(range(1, math.ceil(3976/7))):  # 1~2페이지까지의 블로그 내용을 읽어옴
    url = 'https://section.blog.naver.com/Search/Post.naver?pageNo='+ str(i) +\
    '&rangeType=PERIOD&orderBy=recentdate&startDate=2021-05-04&endDate=2021-08-04&keyword=' + text
    driver.get(url)
    time.sleep(1)
 
    for j in range(1, 8): # 각 블로그 주소 저장(8개로!)
        url_titles = driver.find_element_by_css_selector('#content > section > div.area_list_search > div:nth-child({}) > div > div.info_post > div.desc > a.desc_inner'.format(j))
        url_title = url_titles.get_attribute('href')
        url_list.append(url_title)

        titles=driver.find_element_by_css_selector('#content > section > div.area_list_search > div:nth-child({}) > div > div.info_post > div.desc > a.desc_inner > strong > span'.format(j)).text
        title.append(titles)
        
        names=driver.find_element_by_css_selector('#content > section > div.area_list_search > div:nth-child({}) > div > div.info_post > div.writer_info > a > em'.format(j)).text
        name.append(names)

        days=driver.find_element_by_css_selector('#content > section > div.area_list_search > div:nth-child({}) > div > div.info_post > div.writer_info > span.date'.format(j)).text
        day.append(days)
                       
        
print("url 수집 끝, 해당 url 데이터 크롤링")

df_url = pd.DataFrame()
df_url['url'] = url_list
df_url['titles'] = title
df_url['names'] = name
df_url['days'] = day

df_url['리뷰'] = df_url['url'].apply(lambda x : make_iframe(x))

df_copy = df_url.copy()
df_copy['리뷰'] = df_url.리뷰.apply(lambda x : encode_error(x))
writer = pd.ExcelWriter(r'전기차 후기(21년)-2.xlsx', engine='xlsxwriter',options={'strings_to_urls': False})
df_copy.to_excel(writer)
writer.close()

100%|████████████████████████████████████████████████████████████████████████████████| 567/567 [15:41<00:00,  1.66s/it]


url 수집 끝, 해당 url 데이터 크롤링
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
---

HTTPError: 403 Client Error:  for url: https://blog.naver.com/guyaiydq4032/222442991991

In [97]:
data = pd.read_excel('전기차 후기(19년-1,미완).xlsx')

In [100]:
data['리뷰'] = data['url'].apply(lambda x : make_iframe(x))

------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중

------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중

------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중

------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중

AttributeError: 'NoneType' object has no attribute 'text'

In [88]:
df_url['리뷰'] = df_url['url'].apply(lambda x : make_iframe(x))

------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중

------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중

------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중

------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중 ------
------ 진행중

AttributeError: 'NoneType' object has no attribute 'text'